In [65]:
subject = "606"
run = "2"

In [66]:
%matplotlib inline
import nibabel as nib
img = nib.load('/Users/hellwalker/Desktop/scans/' + subject + '/Pre_Run' + run + '/brnswktm_functional_4.nii.gz')
img_data = img.get_data()
img_data.shape

(96, 114, 96, 600)

In [67]:
#uses AFNI mask to get masked voxels. Gets all Non-Zero voxels.
def get_masked_voxels(mask_data):
    masked_voxels = []
    for x in range(0, mask_data.shape[0]):
        for y in range(0, mask_data.shape[1]):
            for z in range(0, mask_data.shape[2]):
                if mask_data[x, y, z] != 0:
                    masked_voxels.append((x, y, z))
    return masked_voxels

#uses AFNI mask to get masked voxels. Gets voxels with value VALUE.
def get_masked_voxels_specific(mask_data, value):
    masked_voxels = []
    for x in range(0, mask_data.shape[0]):
        for y in range(0, mask_data.shape[1]):
            for z in range(0, mask_data.shape[2]):
                if mask_data[x, y, z] == value:
                    masked_voxels.append((x, y, z))
    return masked_voxels

In [148]:
#Getting transformation from MNI to voxel space
import nibabel as nib
zero_reference_mask = nib.load('/Users/hellwalker/Desktop/scans/zero_reference.nii.gz')
zero_reference_mask_data = zero_reference_mask.get_data()
zero_voxel = get_masked_voxels(zero_reference_mask_data)[0]

def mni2voxel(voxel):
    return (voxel[0] + 2*zero_voxel[0], voxel[1] + 2*zero_voxel[1], voxel[2] + 2*zero_voxel[2])

def voxel2mni(voxel):
    return (voxel[0] - 2*zero_voxel[0], voxel[1] - 2*zero_voxel[1], voxel[2] - 2*zero_voxel[2])

print(mni2voxel((-66, -32, 32)))

(28, 98, 108)


In [124]:
stim_site_mask = nib.load('/Users/hellwalker/Desktop/scans/stim_sites/stim_site_' + subject + '.nii.gz')
stim_site_mask_data = stim_site_mask.get_data()
#stim_site_mask_data.shape
sphere_voxels = get_masked_voxels(stim_site_mask_data)
#print(all_voxels)

In [125]:
"""
#todo: ignore voxels with 0.
sphere_size = 3 #3 voxels in all directions
epicenter = [59, -56, 47]
all_voxels = []#list of lists, with each list being 3D coordinate in sphere
for i in range(-sphere_size, sphere_size+1):
    for j in range(-sphere_size, sphere_size+1):
         for k in range(-sphere_size, sphere_size+1):
                new_voxel = [epicenter[0]+i, epicenter[1]+j, epicenter[2]+k]
                all_voxels.append(new_voxel)
"""

'\n#todo: ignore voxels with 0.\nsphere_size = 3 #3 voxels in all directions\nepicenter = [59, -56, 47]\nall_voxels = []#list of lists, with each list being 3D coordinate in sphere\nfor i in range(-sphere_size, sphere_size+1):\n    for j in range(-sphere_size, sphere_size+1):\n         for k in range(-sphere_size, sphere_size+1):\n                new_voxel = [epicenter[0]+i, epicenter[1]+j, epicenter[2]+k]\n                all_voxels.append(new_voxel)\n'

In [157]:
import copy
import numpy as np
#averaging over all voxels in sphere for every time point
def get_avg_each_timepoint(all_voxels, img_data):
    all_voxels = [list(voxel) for voxel in all_voxels]
    avg_each_timepoint = []
    for t in range(0, img_data.shape[3]):
        all_voxels_copy = copy.deepcopy(all_voxels)
        [all_voxels_copy[voxel].append(t) for voxel in range(0, len(all_voxels_copy))]
        all_data = [img_data[one_voxel[0], one_voxel[1], one_voxel[2], one_voxel[3]] for one_voxel in all_voxels_copy]
        avg = np.average(all_data)
        avg_each_timepoint.append(avg)
    return avg_each_timepoint
avg_each_timepoint = get_avg_each_timepoint(all_voxels, img_data)

In [158]:
import nibabel as nib
mask = nib.load('/Users/hellwalker/Desktop/scans/Morel_Mask_LPI_2mm.nii.gz')
mask_data = mask.get_data()
mask_data.shape

(96, 114, 96)

In [159]:
thalamus_voxels = get_masked_voxels(mask_data)

In [160]:
import scipy as sp
from scipy import stats
def get_correlation_dict(thalamus_voxels, avg_each_timepoint, img_data):
    correlation_dict = {} #thalamus_voxel:correlation between avg_each_timepoint and activity_vector, for finding max activity thalamus voxel
    for thalamus_voxel in thalamus_voxels:
        activity_vector = []
        for t in range(0, img_data.shape[3]):
            activity = img_data[thalamus_voxel[0], thalamus_voxel[1], thalamus_voxel[2], t]
            activity_vector.append(activity)
        correlation_dict[thalamus_voxel] = sp.stats.pearsonr(avg_each_timepoint, activity_vector)[0]
    return correlation_dict
correlation_dict = get_correlation_dict(thalamus_voxels, avg_each_timepoint, img_data)

In [176]:
def get_highest_correlation_key(correlation_dict):
    highest_correlation_key = max(correlation_dict, key=lambda x: correlation_dict[x])
    return highest_correlation_key
best_thalamus_voxel = get_highest_correlation_key(correlation_dict)
print(best_thalamus_voxel, correlation_dict[best_thalamus_voxel])

(43, 50, 40) 0.544384


In [177]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np
def get_4D_graph(best_thalamus_voxel, correlation_dict, sphere_size=4):
    epicenter = best_thalamus_voxel
    all_thalamus_voxels = []#list of lists, with each list being 3D coordinate in sphere
    for i in range(-sphere_size, sphere_size+1):
        for j in range(-sphere_size, sphere_size+1):
             for k in range(-sphere_size, sphere_size+1):
                    new_voxel = (epicenter[0]+i, epicenter[1]+j, epicenter[2]+k)
                    all_thalamus_voxels.append(new_voxel)
    c = []
    for thalamus_voxel in all_thalamus_voxels:
        try:
            c.append(correlation_dict[thalamus_voxel])
        except: #proposed voxel is not in correlation_dict
            c.append(0.0)
    all_thalamus_voxels_array = np.array(all_thalamus_voxels)
    x = all_thalamus_voxels_array[:,0]
    y = all_thalamus_voxels_array[:,1]
    z = all_thalamus_voxels_array[:,2]
    fig = plt.figure()
    from mpl_toolkits.mplot3d import Axes3D
    ax = fig.add_subplot(111, projection='3d')
    plt.title("correlations of ± " + str(sphere_size) + " voxels about " + str(epicenter))
    ax.set_xlabel('Left to Right')
    ax.set_ylabel('Posterior to Anterior')
    ax.set_zlabel('Inferior to Superior')
    sp = ax.scatter(x, y, z, c=c, depthshade=True)
    plt.colorbar(sp)
    fig.tight_layout()
    plt.savefig('output_4D.png', dpi=300)
#get_4D_graph(best_thalamus_voxel, correlation_dict)

In [178]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np

#f(2) --> 1 (2 - 1 = 1), 0 (2 - 2 = 0)
#f(1) --> 0 (1 - 1 = 0), 2 (1 - 2 = -1 = 2)
#f(0) --> 2 (0 - 1 = -1 = 2), 1 (0 - 2 = -2 = 1)

def get_volcano_graph(all_thalamus_voxels, best_thalamus_voxel, correlation_dict, flip=False, axis='z'):
    if axis == 'z':
        axis_num = 2
    elif axis == 'y':
        axis_num = 1
    elif axis == 'x':
        axis_num = 0
    else:
        raise NameError("Specify 'x', 'y', or 'z' for the fourth argument")
    
    z_plane = best_thalamus_voxel[axis_num]
    thalamus_voxels_on_plane = [voxel for voxel in all_thalamus_voxels if voxel[axis_num] == z_plane]
    thalamus_voxels_on_plane_array = np.array(thalamus_voxels_on_plane)
    x = thalamus_voxels_on_plane_array[:,axis_num-2]
    y = thalamus_voxels_on_plane_array[:,axis_num-1]
    z = []
    for thalamus_voxel in thalamus_voxels_on_plane:
        try:
            z.append(correlation_dict[thalamus_voxel])
        except: #proposed voxel is not in correlation_dict
            z.append(0.0)
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    
    if flip:
        ax.plot_trisurf(y, x, z, cmap=cm.jet, linewidth=0.05)
    else:
        ax.plot_trisurf(x, y, z, cmap=cm.jet, linewidth=0.05)
    title = "correlations of thalamus voxels at " + axis + "=" + str(best_thalamus_voxel[axis_num]) + " with max " +  str(best_thalamus_voxel)
    if flip:
        title += " flipped"
    plt.title(title)
    #ax.set_xlabel('Left to Right')
    #ax.set_ylabel('Posterior to Anterior')
    ax.set_zlabel('Correlation')
    plt.savefig(title + ".png", dpi=300)
    plt.show()
    
#get_volcano_graph(thalamus_voxels, best_thalamus_voxel, correlation_dict, True, 'x')
#get_volcano_graph(thalamus_voxels, best_thalamus_voxel, correlation_dict, False, 'x')
#get_volcano_graph(thalamus_voxels, best_thalamus_voxel, correlation_dict, True, 'y')
#get_volcano_graph(thalamus_voxels, best_thalamus_voxel, correlation_dict, False, 'y')
#get_volcano_graph(thalamus_voxels, best_thalamus_voxel, correlation_dict, True, 'z')
#get_volcano_graph(thalamus_voxels, best_thalamus_voxel, correlation_dict, False, 'z')

In [179]:
best_voxel_sphere_mask = nib.load('/Users/hellwalker/Desktop/scans/thalamus_sites/thalamus_site_' + subject + '_Run' + run + '.nii.gz')
best_voxel_sphere_mask_data = best_voxel_sphere_mask.get_data()
print(best_voxel_sphere_mask_data.shape)
#print(best_thalamus_voxel)
sphere_voxels = get_masked_voxels(best_voxel_sphere_mask_data)
#print(sphere_voxels)
#print(thalamus_voxels)
"""The best thalamus voxel may be at the edge of the thalamus. Therefore, the sphere may
contain voxels that are not in the thalamus. Therefore, sphere_voxels that are not in 
thalamus_voxels will be filtered out."""
filtered_sphere_voxels = [list(voxel) for voxel in sphere_voxels if voxel in thalamus_voxels]
#print(filtered_sphere_voxels)

(96, 114, 96)


In [180]:
best_voxel_sphere_activity_vector = avg_each_timepoint
#best_voxel_sphere_activity_vector = get_avg_each_timepoint(filtered_sphere_voxels, img_data)
#print(best_voxel_sphere_activity_vector)

In [121]:
import nibabel as nib
gordon_mask = nib.load('/Users/hellwalker/Desktop/scans/Gordon_333_cortical_2mm.nii.gz')
gordon_mask_data = gordon_mask.get_data()
#gordon_mask_data.shape
#ROI_dict = {i:get_masked_voxels_specific(gordon_mask_data, i) for i in range(1, 334)}
#print(ROI_dict)

In [122]:
#creating ROI_dict takes forever. Therefore, I have serialized the object, and it can be deserialized if needed.
import pickle
#ROI_dict_file = open("/Users/hellwalker/Desktop/scans/ROI_dict_file", "wb") #create a new file 
#pickle.dump(ROI_dict, ROI_dict_file, 2)
ROI_dict_file = open("/Users/hellwalker/Desktop/scans/ROI_dict_file", "rb")
ROI_dict = pickle.load(ROI_dict_file)
#print(len(ROI_dict[333]))

48


In [170]:
"""ROI_activity_dict is a dictionary of {ROI number: activity vector},
where activity vector consists of the average of the activity across the ROI at each timepoint.
This is also serialized to save time."""
#ROI_activity_dict = {i: get_avg_each_timepoint(ROI_dict[i], img_data) for i in range(1, 334)}
#ROI_activity_dict_file = open("/Users/hellwalker/Desktop/scans/ROI_dict_activity_" + subject + "_Run" + run, "wb") #create a new file 
#pickle.dump(ROI_activity_dict, ROI_activity_dict_file, 2)
ROI_activity_dict_file = open("/Users/hellwalker/Desktop/scans/ROI_dict_activity_" + subject + "_Run" + run, "rb")
ROI_activity_dict = pickle.load(ROI_activity_dict_file)
#avg_each_timepoint = get_avg_each_timepoint(ROI_dict[333], img_data)
#print(avg_each_timepoint)
#print(ROI_activity_dict)

In [194]:
import operator
#produces correlation dictionary with {ROI number: correlation with best_thalamus_voxel}
ROI_correlation_dict = {i: sp.stats.pearsonr(best_voxel_sphere_activity_vector, ROI_activity_dict[i])[0] for i in range(1, 334)}
#print(ROI_correlation_dict)
#print(get_highest_correlation_key(ROI_correlation_dict), ROI_correlation_dict[63])
sorted_dict = sorted(ROI_correlation_dict.items(), key=operator.itemgetter(1))
best_20_with_corr = sorted_dict[-20:]
best_20 = [item[0] for item in best_20_with_corr]
print(best_20)

[195, 42, 193, 85, 109, 208, 228, 177, 229, 100, 230, 41, 76, 23, 231, 62, 192, 103, 105, 63]


In [ ]:
#do the same thing with stimulation site. the only difference would be the value of avg_each_timepoint